In [ ]:
!pip install timm
import torch
import torch.nn as nn
from torchvision import transforms
import timm
import unicodedata
import cv2

def data_load(input_path):
  data = cv2.imread(input_path)
  data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
  data = cv2.resize(data, (224,224))
  data = Image.fromarray(data)

  transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])

  data = transform(data)
  return data.unsqueeze(dim=0)

input_path = './사람1.png'
keyword = '대구 팔공산 갓바위'

# keyword = unicodedata.normalize('NFC', keyword)

model = timm.create_model('efficientnet_b0', pretrained = True)
model.classifier = nn.Sequential(
    nn.Linear(1280,512),
    nn.ReLU(),
    nn.Linear(512,128),
)
checkpoint = torch.load('./best_model.pth')
model.load_state_dict(checkpoint)

pickle_filename = "image_test_dict.pickle"
with open(pickle_filename, 'rb') as handle:
    image_test_dict = pickle.load(handle)

data = data_load(input_path)

model.eval()
output1 = image_test_dict[keyword]
output2 = model(data)
n = output1.shape[0]
output2 = torch.cat([output2]*n)

count = sum(torch.pow(output1 - output2, 2).sum(dim=1).sqrt() < 0.68)
result = (count >= 1).int()

result.item()